In [28]:
import pandas as pd
import json
import numpy as np
import glob
import pickle

In [29]:
ta = pd.read_excel('Tourist_Attraction.xlsx')

In [33]:
ta_part = ta[["微博id","微博主页","文本","日期","图片","转发数","评论数","点赞数","地点","FID","经度","纬度"]]

In [37]:
ta_part = ta_part[:14505]

In [38]:
new_lines = []

for row in ta_part.to_numpy():
    imgs = row[4].split()
    empty_row = row
    empty_row[4] = ''
    for img in imgs:
        empty_row[4] = img
        new_lines.append(empty_row)

In [42]:
new_pds = pd.DataFrame(new_lines, columns=["微博id","微博主页","文本","日期","图片","转发数","评论数","点赞数","地点","FID","经度","纬度"])

In [79]:
pic_reg = "data/Tourist_Attraction_Pickle/*.pickle"
pics = glob.glob(pic_reg)

img2supcls = {}

for pic_path in tqdm(pics, total=len(pics), ncols=50):
    imgid = pic_path.split('/')[-1].split('.')[0]
    # if imgid == '7546503fly1g9hidlypomj218y0u0qfk':
    with open(pic_path, 'rb') as f:
        this_result = pickle.load(f)
        # draw_panoptic_segmentation(**this_result)

        # print(this_result)
        segmentation = this_result['segmentation'].cpu().numpy()
        seg_info = this_result['segments_info']
        ids = np.array([i['id'] for i in seg_info])
        idxs = np.array([i['label_id'] for i in seg_info])
        id2idxs = {i['id']: i['label_id'] for i in seg_info}

        id2counts = {i:np.sum(segmentation==i) for i in ids}
        idx2counts = {i:0 for i in np.unique(idxs)}
        for i in id2counts:
            idx = id2idxs[i]
            idx2counts[idx] += id2counts[i]

        this_idxs = np.unique(idxs)
        this_counts = np.array([idx2counts[i] for i in this_idxs])

        # print(this_counts, np.flip(np.argsort(this_counts)))
        sort_ids = this_idxs[np.flip(np.argsort(this_counts))]
        # print(sort_ids)
        subclss = [idxs2lab[str(i)] for i in sort_ids]

        # sort_idxs = [id2idxs[i] for i in sort_ids]
        # print(sort_ids)

        img2supcls[imgid] = subclss

100%|█████| 70005/70005 [00:38<00:00, 1795.44it/s]


In [83]:
new_pds.shape

(77924, 12)

In [78]:
img2supcls

{'7546503fly1g9hidlypomj218y0u0qfk': ['building-other-merged',
  'person',
  'floor-other-merged',
  'banner']}

In [47]:
with open('/home/yangwenhao/.cache/huggingface/hub/models--facebook--mask2former-swin-large-coco-panoptic/snapshots/85b535928a783691eaf27467a573b26d543336ea/config.json', 'r') as f:
    configs = json.load(f)
    
idxs2lab = configs['id2label']

In [51]:
lab2idxs = {idxs2lab[i]:i for i in idxs2lab}

In [61]:
subcls2supcls = {}
with open('supcls2subcls.txt', 'r') as f:
    for l in f.readlines():
        supcls, subclss = l.split("：")
        subclss = subclss.rstrip(' |\n')
        for subcls in subclss.split(','):
            subcls = subcls.rstrip(' ')
            subcls = subcls.lstrip(' ')
            subcls2supcls[subcls] = supcls

In [82]:
len(img2supcls)

70005

In [84]:
tag1 = []
tag2 = []
tag3 = []
tag4 = []
miss_imgs = []
valid_imgs = set(img2supcls.keys())

for i in new_pds.图片.to_list():
    imgid = i.split('.')[0]
    if imgid not in valid_imgs:
        miss_imgs.append(imgid)
        img_clss = []
    else:
        img_clss = img2supcls[imgid]
    if len(img_clss) > 0:
        tag1.append(img_clss[0])
    else:
        tag1.append('')
        
    if len(img_clss) > 1:
        tag2.append(img_clss[1])
    else:
        tag2.append('')

    if len(img_clss) > 2:
        tag3.append(img_clss[2])
    else:
        tag3.append('')
        
    if len(img_clss) > 3:
        tag4.append(' '.join(img_clss[3:]))
    else:
        tag4.append('')
        

In [85]:
miss_imgs

['9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv',
 '9cf48bcfly1g6mjpghn7uj20u00k0dkv']

In [87]:
img_lst = new_pds.图片.to_list()
print(len(img_lst), len(np.unique(img_lst)))

77924 13273
